# Mini Projekt - Baby Kyber

## Pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$

In [ ]:
# skopiuj pierścień ilorazowy wielomianów z pierwszych zajęć
import numpy as np
from copy import deepcopy



def create_poly(table):
    try:
        table[0]
        return Poly(table)
    except:
        if isinstance(table, Poly):
            return table
        return Poly([table])

class Poly:
    def __init__(self, coef, variable = None):
        self.coef = coef
        self.variable = variable
        if self.variable is None : self.variable = 'x'
        if len(self.variable) > 1 : self.variable = f"({self.variable})"

    def normalize(self) :
        while self.coef[-1] == 0 and len(self.coef) > 1 :
            self.coef.pop()

    def __iter__(self):
        self.current = 0
        return self

    def __next__(self):
        if self.current < len(self.coef):
            item = self.coef[self.current]
            self.current += 1
            return item
        else:
            raise StopIteration


    def __getitem__(self, index):
        if isinstance(index, slice) :
            return Poly(self.coef[index], self.variable)
        return self.coef[index]

    def __setitem__(self, index, value):
        if isinstance(index, slice) :
            if isinstance(value, Poly):
                self.coef[index] = value.coef
            else:
                self.coef[index] = value
        else:
            self.coef[index] = value

    def __len__(self):
        return len(self.coef)
    
    def __add__(self, poly2) :
        return Poly(np.polyadd(self.coef, poly2.coef))
    
    def __mul__(poly1, poly2) :               
        return Poly(np.polymul(poly1.coef, poly2.coef))
    
    def __rmul__(self, other):
        return self * other
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other) :
        return self - other
    
    def __sub__(poly1, poly2) :
        return poly1 + (poly2 * -1)
    
    def __truediv__(poly1, poly2) :
        poly1 = create_poly(poly1)
        poly2 = create_poly(poly2)
        
        poly = deepcopy(poly1)
            
        output = Poly([0] * (len(poly1) - len(poly2) + 1))
        i = len(poly1) - 1
        for i in range(len(poly1) - 1, len(poly2) - 2, -1) :
            index = i - len(poly2) + 1
            output[index] = poly[i] / poly2[-1]
            poly = poly - (poly2 * output[:index + 1])
        
        return output, poly
            
    def __floordiv__(poly1, poly2) :
        return (poly1 / poly2)[0]
    
    def __mod__(poly1, poly2) :


FIXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

        poly1 = create_poly(poly1)
        poly2 = create_poly(poly2)
        
        poly = deepcopy(poly1)
            
        output = Poly([0] * (len(poly1) - len(poly2) + 1))
        i = len(poly1) - 1
        for i in range(len(poly1) - 1, len(poly2) - 2, -1) :
            index = i - len(poly2) + 1
            output[index] = poly[i] / poly2[-1]
            poly = poly - (poly2 * output[:index + 1])
        
        return poly
    
    def __eq__(poly1, poly2) :
        if not (isinstance(poly1, Poly) and isinstance(poly2, Poly)) : return False
        check1 = deepcopy(poly1)
        check2 = deepcopy(poly2)
        check1.normalize()
        check2.normalize()
        
        if len(check1) != len(check2) : return False

        for i in range(len(check1)) :
            if check1[i] != check2[i] : return False

        return True        
            
    def convert_to_nice(self,val,i):
        if val == 0:
            return ""
        if self.first_print:
            symbol = ""
            self.first_print = False
        else:
            symbol = " - " if val < 0 else " + "

        
        if len(self.coef) - i - 1 == 0:
            return symbol + str(abs(val))
        
        result = f"x^{len(self.coef) - i - 1}"


        if abs(val) == 1:
            return symbol + result
        else:
            return symbol + str(val) + result

    def __str__(self):
        self.first_print = True
        return f'{"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.coef))}'
    def __repr__(self):
        self.first_print = True
        return f'{"".join(self.convert_to_nice(val,i) for i, val in enumerate(self.coef))}'


class ZnWRing:
    def __init__(self, N, W):
        self.N = N
        self.W = self.moderate(W)

    def fix_float(self, number) :
        return float(number) if not float(number).is_integer() else int(number)
    
    def moderate(self, poly) :
        return Poly([self.fix_float(x % self.N) for x in poly])
    
    def repair(self, poly) :
        
        if isinstance(poly, int) or isinstance(poly, float) : poly = poly % self.N
        else : poly = self.moderate((poly % self.W))

        poly2 = []
        for number in poly:
            poly2.append(self.fix_float(number))
        return poly2
        
    def add(self, poly1, poly2) :
        return self.repair(poly1 + poly2)
    
    def subtract(self, poly1, poly2) :
        return self.repair(poly1 - poly2)
    
    def mul(self, poly1, poly2) :
        return self.repair(poly1 * poly2)

    def div(self, poly1, poly2) :
        return self.repair(poly1 / poly2)
    
    
    def __str__(self) :
        return repr(self)
    
    def __repr__(self) :
        return f"N = {self.N}\nW = " + str(self.W)
    
    def __eq__(self, other):
        if not isinstance(other, ZnWRing):
            return ValueError("Must be a ZnWRing!")
        return self.N == other.N and self.W == other.W




class ZnPoly:
    def __init__(self, poly, N = 17, W = [1,0,0,0,1]):
        self.ZnWRing = ZnWRing(N,W)
        self.poly = self.ZnWRing.moderate(create_poly(poly))
    
    def __mul__(self,other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.mul(self.poly,other.poly))

    def __rmul__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return self.__mul__(other)
    
    def __add__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(self.poly, other.poly))
    
    def __radd__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return self.__add__(other)
    
    def __sub__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(self.poly, other.poly))
    
    def __rsub__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly(self.ZnWRing.add(other.poly, self.poly))

    def __len__(self):
        return len(self.poly)

    def __eq__(self, other):
        if not isinstance(other, ZnPoly):
            return ValueError("Must be a ZnPoly!")
        return ZnPoly.__eq__(self.poly, other.poly) and self.ZnWRing == other.ZnWRing
    
    def __repr__(self):
        return str(self.poly)
    
    def __str__(self):
        return self.__repr__()


w1 = [7,0,0,14,0,0,0]
w2 = [0,12,10,6]
w3 = [1,1,11,7]

w1 = ZnPoly(w1)
w2 = ZnPoly(w2)
w3 = ZnPoly(w3)

print(f"w1 = {w1}")
print(f"w2 = {w2}")
print(f"w3 = {w3}")

print(f"w1 + w2 = {w1 + w2}")
print(f"w1 * w2 = {w1 * w2}")

w1 = 7x^6 + 14x^3
w2 = 12x^2 + 10x^1 + 6
w3 = x^3 + x^2 + 11x^1 + 7


AttributeError: 'int' object has no attribute 'coef'

## Baby Kyber

Zaimplementuj poniższe elementy kryptosystemu Baby Kyber tak, aby osiągnąć jak największą skuteczność w testach (przy niezerowych błędach). Wymagana minimalna skuteczność to 60%.

In [25]:
K = 2
q = 17

#pierścień Z17[X]/X^4 + 1

prop = 0.1

n = 4

### Generowanie klucza

Zaimplementuj funkcję `key_gen()` realizującą generowanie klucza w kryptosystemie Baby Kyber. Funkcja ma zwracać `A,t,s`. Przetestuj, czy dla podanych $A,s,e$ otrzymasz poprawny wielomian $t$.

$A=\left[\begin{matrix}
    6x^3+16x^2+16x+11&9x^3+4x^2+6x+3\\
    5x^3+3x^2+10x+1&6x^3+x^2+9x+15
\end{matrix}\right]$

$\mathbf{s}=(-x^3-x^2+x,-x^3-x)$

$\mathbf{e}=(x^2,x^2-x)$

$\mathbf{t}=A\mathbf{s}+\mathbf{e}:\ \ \mathbf{t}=(16x^3+15x^2+7,10x^3+12x^2+11x+6)$

In [45]:
def B_ni_1(prop):
    random_value = np.random.uniform(0,1)

    if random_value < prop:
        return -1
    if random_value < 2*prop:
        return 1
    
    return 0

def matrix_vector_multiply(A, B):
    rows_A = len(A)
    cols_A = len(A[0])
    
    if cols_A != len(B):
        raise ValueError("Number of columns in A must equal the size of vector B.")
    
    result = [ZnPoly([0]) for _ in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_A):
            result[i] = result[i] + (A[i][j] * B[j])
    
    return result

def key_gen():
    def generate_A():
        return [[ZnPoly(np.random.randint(0,q,size=n)) for _ in range(K)] for _ in range(K)]
    
    def generate_s():
        return [ZnPoly([B_ni_1(prop) for _ in range(n)]) for _ in range(K)]

    
    def generate_e():
        return generate_s()
        

    def calculate_t(A,s,e):
        t = [ZnPoly([0] * n) for _ in range(K)]
        for i in range(K):
            for j in range(K):
                t[i] = t[i] + A[i][j] * s[j]
            t[i] = t[i] + e[i]
        return t
    
    A = generate_A()
    s = generate_s()
    e = generate_e()
    t = calculate_t(A,s,e)

    return A,s,t

def test_calculate_t(A,s,e):
    t = matrix_vector_multiply(A,s)
    print(t)
    return t + e


A = np.array([  [ZnPoly([6,16,16,11]), ZnPoly([9,4,6,3])],
                [ZnPoly([5,3,10,11]),  ZnPoly([6,1,9,15])]])

s = np.array([ZnPoly([-1,-1,1,0]), ZnPoly([-1,0,-1,0])])

e = np.array([ZnPoly([0,1,0,0]), ZnPoly([0,1,-1,0])])

t = test_calculate_t(A,s,e)

print("TEST FOR THE EXAMPLE")
print(t)

print()


A,s,t = key_gen()

# print(s)
# print(t)
# print(A)

[16x^3 + 14x^2 + 7, x^2 + 5x^1 + 6]
TEST FOR THE EXAMPLE
[16x^3 + 15x^2 + 7 2x^2 + 4x^1 + 6]



### Szyfrowanie

Zaimplementuj funkcję `encrypt(A,t,m)` realizującą szyfrowanie w kryptosystemie Baby Kyber a gdzie wejściowe `m` jest w postaci listy. Funkcja ma zwracać szyfrogram `c`. Przetestuj poprawność działania na poniższych danych. 

$m=1\cdot x^3+0\cdot x^2+1\cdot x+1=x^3+x+1$

$\mathbf{r}=(-x^3+x^2,x^3+x^2-1)$

$\mathbf{e_1}=(x^2+x,x^2)$

$e_2=-x^3-x^2$

$\mathbf{u}=A^T\mathbf{r}+\mathbf{e_1}:\ \ \mathbf{u}=(11x^3+11x^2+10x+3,4x^3+4x^2+13x+11)$

$v=\mathbf{t}^T\mathbf{r}+e_2+\lfloor\frac{q}{2}\rceil m:\ \ v=8x^3+6x^2+9x+16$

$\mathbf{c}=(\mathbf{u},v):\ \ \mathbf{c}=((11x^3+11x^2+10x+3,4x^3+4x^2+13x+11),8x^3+6x^2+9x+16)$

In [ ]:
def rand_small_polynomial(self):
    """
    Generate a random 'small' polynomial
    """
    SMALL_NUMBER_LIMIT = 1
    coef = []
    for i in range(self.R.n):
        coef.append(np.random.randint(-SMALL_NUMBER_LIMIT, SMALL_NUMBER_LIMIT))
    return self.R(coef)

def encrypt(A,t,m):
    m = Polynomial(m)
    


### Deszyfrowanie

Zaimplementuj funkcję `decrypt(c,s)` realizującą deszyfrowanie w kryptosystemie Baby Kyber. Funkcja ma zwracać ostateczną odszyfrowaną wiadomość `m_n`. Przetestuj działanie na poniższych danych.

$m_n=v-\mathbf{s}^T\mathbf{u}:\ \ m_n=8x^3+14x^2+8x+6$

$m_n=1\cdot x^3+0\cdot x^2+1\cdot x+1$


### Testy

In [ ]:
import secrets as sc

success = 0
for i in range(1000):
    output = []
    A,t,s = key_gen()
    
    m=[sc.choice((0,1)) for k in range(4)]
    
    c = encrypt(A,t,m)
    m_n = decrypt(c,s)

    if m_n == m:
        success += 1

print(f'Success rate: {success * 100 /1000} %')
